# Travel Hotel Recommendation APP design with Streamlit

### Import and install Library

In [4]:
!pip install streamlit
!pip install flask

  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)

   ---------------------------------------- 0/3 [werkzeug]
   ---------------------------------------- 0/3 [werkzeug]
   ---------------------------------------- 0/3 [werkzeug]
   ------------- -------------------------- 1/3 [itsdangerous]
   -------------------------- ------------- 2/3 [flask]
   ---------------------------------------- 3/3 [flask]



In [5]:
from flask import Flask, request,render_template
import pandas as pd
from sklearn.compose import ColumnTransformer
import numpy as np
import pickle
import os
import streamlit as st

### Development of ML code

In [ ]:
path = 'D:/AlmaBetter/P01_travelPrice/data/hotels.csv'
df = pd.read_csv(path,)
df.head()

,code,company,name,gender,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44


In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load the dataset
data = df

# Data Preprocessing
# Combine relevant columns into a single column for hotel information
data['Hotel_Info'] = data['name'].str.cat(data['place'], sep='|')

# Create a TF-IDF vectorizer to convert text data into numerical vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the vectorizer on the Hotel_Info column
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Hotel_Info'])

# Compute the cosine similarity between hotels based on TF-IDF vectors
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get hotel recommendations based on Package Type, Start City, Price, and Destination
def get_hotel_recommendations(place, days, price, total, cosine_sim=cosine_sim):
    # Filter the dataset based on the given criteria
    filtered_data = data[(data['place'] == place) &
                         (data['days'] <= days) &
                         (data['price'] <= price) &
                         (data['total'] <= total)]

    if filtered_data.empty:
        return "No matching hotels found."

    # Get the indices of the filtered hotels
    hotel_indices = filtered_data.index

    # Calculate the average cosine similarity score for each hotel with the filtered hotels
    avg_similarity_scores = []
    for idx in hotel_indices:
        avg_score = sum(cosine_sim[idx]) / len(cosine_sim[idx])
        avg_similarity_scores.append(avg_score)

    # Create a DataFrame to store the filtered hotels and their average similarity scores
    recommended_hotels_df = pd.DataFrame({'travelCode': filtered_data['travelCode'],
                                          'Hotel Details': filtered_data['name'],
                                          'Avg Similarity Score': avg_similarity_scores})

    # Sort the hotels by average similarity score in descending order
    recommended_hotels_df = recommended_hotels_df.sort_values(by='Avg Similarity Score', ascending=False)

    # Return the recommended hotel details
    return recommended_hotels_df[['travelCode', 'Hotel Details']]


KeyError: 'place'

### Streamlit app design